In [7]:
%pylab nbagg
from tvb.simulator.lab import *
import os.path
from matplotlib import colors, cm
import time
import scipy.signal as sig

import numpy as np
import pandas as pd
import scipy.io

# downloaded library for peak detection in z time series
import peakdetect

%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
from runmainsim import *

In [35]:
patients = ['id001_ac', 'id002_cj', 'id014_rb']
for patient in patients:
#     patient='cj'
    # 1000 = 1 second
    samplerate = 1000 # Hz
    sim_length = 60*samplerate

    ezregion = []
    pzregion = []

    x0norm=-2.3
    x0ez=-1.8
    x0pz=2.05

    root_dir = os.getcwd()
    project_dir = os.path.join(root_dir, "metadata/"+patient)
    print "Project directory for meta data is : ", project_dir

    confile = os.path.join(project_dir, "connectivity.zip")

    # convert seeg.xyz to seeg.txt file
    sensorsfile = os.path.join(project_dir, "seeg.xyz")
    newsensorsfile = os.path.join(project_dir, "seeg.txt")
    try:
        os.rename(sensorsfile, newsensorsfile)
    except:
        print "Already renamed seeg.xyz possibly!"

    # convert gain_inv-square.mat file into gain_inv-square.txt file
    gainmatfile = os.path.join(project_dir, "gain_inv-square.mat")
    newgainmatfile = os.path.join(project_dir, "gain_inv-square.txt")
    try:
        os.rename(gainmatfile, newgainmatfile)
    except:
        print "Already renamed gain_inv-square.mat possibly!"

    sim, configs = initmodel(confile, newsensorsfile, newgainmatfile, ezregion=ezregion, pzregion=pzregion, x0norm=x0norm, x0ez=x0ez, x0pz=x0pz)
    times, epilepts, seegts = runsim(sim, sim_length)

    # save data now from output and get relevant data
    display(configs)

    outputdir = os.path.join(root_dir, 'output')
    filename = os.path.join(outputdir, patient+'_sim_nez'+str(len(ezregion))+\
                                        '_npz'+str(len(pzregion))+'.npz')

    ######################## POST PROCESSING #################################
    # post process by cutting off first 5 seconds of simulation
    times, epits, seegts, zts = postprocts(epilepts, seegts, times)

    ezindices, pzindices = getindexofregion(configs.connectivity.region_labels, ezregion, pzregion)
    # get the onset, offset times
    onsettimes = None
    offsettimes = None
    try:
        onsettimes, offsettimes = findonsetoffset(zts[:, ezindices].squeeze())
    except:
        print "Still not working..."

    ######################## SAVING ALL DATA #################################
    regions = configs.connectivity.region_labels
    # Save files
    meta = {
        'x0ez':x0ez,
        'x0pz':x0pz,
        'x0norm':x0norm,
        'ez': regions[ezindices],
        'pz': regions[pzindices],
        'ezindices': ezindices,
        'pzindices': pzindices,
        'onsettimes':onsettimes,
        'offsettimes':offsettimes,
        'patient':patient
    }

    # save tseries
    np.savez(filename, epits=epits, seegts=seegts, \
             times=times, zts=zts, metadata=meta)

Project directory for meta data is :  /Users/adam2392/Documents/tvb/metadata/id002_cj
Already renamed seeg.xyz possibly!
Already renamed gain_inv-square.mat possibly!


initial_conditions,None
coupling,Difference(a=1)
stimulus,None
integrator,"HeunStochastic(dt=0.04, noise=Additive(dt=0.04, ntau=0))"
surface,None
connectivity,"Connectivity(bound=False, value=None)"
conduction_speed,inf
simulation_length,5000
model,"Epileptor(bound=False, value=None)"
monitors,"[TemporalAverage(bound=False, value=None), iEEG(bound=False, value=None)]"
